# Dynamically route logic based on input(根据输入动态路由决策)

本笔记本涵盖了如何在LangChain表达语言中进行路由。
路由允许您创建非确定性链，其中前一步的输出定义了下一步。路由有助于为与LLMs的交互提供结构和一致性。
执行路由有两种方式：
1. 使用RunnableBranch。
2. 编写自定义工厂函数，该函数接受上一步骤的输入并返回一个可运行对象。重要的是，这应该返回一个可运行对象，并且实际上不执行。

我们将使用两个步骤序列来说明这两种方法，在第一步中对输入问题进行分类为关于LangChain、Anthropic或Other，然后将其路由到相应的提示链。

### Using a RunnableBranch

一个RunnableBranch是用一组（条件，可运行）对和一个默认的可运行初始化的。它通过将每个条件传递给其调用时的输入来选择哪个分支。它选择第一个评估为True的条件，并使用输入运行与该条件相对应的可运行。

如果没有提供的条件匹配，则运行默认的可运行。

以下是其实际操作示例：

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 创建AI模型
model = ChatOpenAI(
)

首先，让我们创建一个链条，用来识别传入的问题是关于LangChain、Anthropic还是其他的。

In [4]:
chain = (
    PromptTemplate.from_template(
        """Given the user question below, classify it as either being about `LangChain`, `OpenAI`, or `Other`.

Do not respond with more than one word.

<question>
{question}
</question>

Classification:"""
    )
    | model
    | StrOutputParser()
)

In [5]:
chain.invoke({"question": "how do I call OpenAI?"})

'OpenAI'

现在，让我们创建三个子链：

In [6]:
langchain_chain = (
  PromptTemplate.from_template(
        """你是一个Langchain方面的专家. \
问题的答案总是以 "在Langchain官方文档中我了解到" 开始. \
回答下面的问题:

问题: {question}
答案:"""
    ) | model
)

openAI_chain = (
    PromptTemplate.from_template(
        """你是一个 OpenAI 方面的专家. \
问题的答案总是以 "在OpenAI官方文档中我了解到" 开始.. \
回答下面的问题:

问题: {question}
答案:"""
    )
    | model
)

general_chain = (
    PromptTemplate.from_template(
        """回答下面的问题:

问题: {question}
答案:"""
    )
    | model
)

In [7]:
from langchain_core.runnables import RunnableBranch

branch = RunnableBranch(
    (lambda x: "openai" in x["topic"].lower(), openAI_chain),
    (lambda x: "langchain" in x["topic"].lower(), langchain_chain),
    general_chain,
) 

In [8]:
full_chain = {"topic": chain, "question": lambda x: x["question"]} | branch | StrOutputParser()

In [9]:
full_chain.invoke({"question": "我如何使用OpenAI?"})

'在OpenAI官方文档中我了解到，要使用OpenAI，首先需要注册一个OpenAI帐户并获取API密钥。然后，可以使用API密钥来访问OpenAI的各种服务和功能，如文本生成、语言处理等。具体的使用方法和代码示例可以在OpenAI的官方文档中找到。'

In [10]:
full_chain.invoke({"question": "我如何使用Langchain?"})

'在Langchain官方文档中我了解到，要使用Langchain，首先需要下载并安装Langchain的客户端软件。然后，您可以创建一个新的账户并开始使用Langchain的区块链功能，包括发送和接收加密货币，参与智能合约等。您还可以使用Langchain的开发工具来构建基于Langchain的应用程序。'

In [11]:
full_chain.invoke({"question": "1+1等于多少?"})

'1+1=2'

### 使用自定义函数

您还可以使用自定义函数来在不同输出之间进行路由。以下是一个示例：


In [12]:
# 路由决策函数
def route(info):
  if "openai" in info["topic"].lower():
    return openAI_chain
  elif "langchain" in info["topic"].lower():
    return langchain_chain
  else:
    return general_chain
  
from langchain_core.runnables import RunnableLambda

full_chain = {"topic":chain,"question":lambda x: x["question"]} | RunnableLambda(route) | StrOutputParser()

full_chain.invoke({"question": "我如何使用OpenAI?"})

'在OpenAI官方文档中我了解到，您可以使用OpenAI的API来访问其各种人工智能模型和工具。您可以通过申请API密钥并遵循其文档中的指南来开始使用OpenAI的功能。'